# Retrieval Augmented Generation (RAG) with [LangChain](https://python.langchain.com/docs)

## 1. Install Dependencies

In [ ]:
%pip install langchain==0.0.345 openai==1.2.3 pypdf chromadb pysqlite3-binary tiktoken

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

## 2. Set API access

You may choose from a wide array of [providers](https://python.langchain.com/docs/integrations) to integrate with. The example below uses OpenAI models through [Azure](https://oai.azure.com/portal).

In [ ]:
%env AZURE_OPENAI_ENDPOINT=https://<your-endpoint>.openai.azure.com/
%env AZURE_OPENAI_API_KEY=<YOUR_AZUREOPENAI_KEY>
%env OPENAI_API_TYPE=azure
%env OPENAI_API_KEY=<YOUR_AZUREOPENAI_KEY>
%env OPENAI_API_VERSION=2023-05-15

## 3. Read data

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load the text of the Geneva Convention.
loader = PyPDFLoader("https://ihl-databases.icrc.org/assets/treaties/365-GC-I-EN.pdf")
pages = loader.load_and_split()


## 4. Create vector store retriever

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import AzureOpenAIEmbeddings
from chromadb.config import Settings

deployment_name = "<your-embedding-deployment-name>"
embeddings = AzureOpenAIEmbeddings(azure_deployment=deployment_name)
vectorstore = Chroma.from_documents(
    documents=pages,
    embedding=embeddings,
    client_settings=Settings()
)
retriever = vectorstore.as_retriever()

## 5. Create model

In [ ]:
from langchain.chat_models import AzureChatOpenAI

temperature = 0.2
deployment_name = "<your-llm-deployment-name>"
llm = AzureChatOpenAI(temperature=temperature, azure_deployment=deployment_name)

## 6. Perform Q&A

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt_template = PromptTemplate.from_template("{question}")
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

question = "What is the emblem of the Geneva Convention?"
rag_chain.invoke(question)